In [ ]:
import torch
from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType,
    get_default_template_type, inference_stream
)
from swift.utils import seed_everything

#モデル初期化
model_type = ModelType.qwen2_vl_2b_instruct
template_type = get_default_template_type(model_type)
model, tokenizer = get_model_tokenizer(model_type, torch.bfloat16,
                                       model_kwargs={'device_map': 'auto'})
model.generation_config.max_new_tokens = 256
template = get_template(template_type, tokenizer)
seed_everything(42)


/home/setup/miniconda3/envs/qwenft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[INFO:swift] Successfully registered `/home/setup/miniconda3/envs/qwenft/lib/python3.10/site-packages/swift/llm/data/dataset_info.json`
[INFO:swift] No vLLM installed, if you are using vLLM, you will get `ImportError: cannot import name 'get_vllm_engine' from 'swift.llm'`
[INFO:swift] No LMDeploy installed, if you are using LMDeploy, you will get `ImportError: cannot import name 'prepare_lmdeploy_engine_template' from 'swift.llm'`
[INFO:swift] Downloading the model from ModelScope Hub, model_id: qwen/Qwen2-VL-2B-Instruct


[WARNING:modelscope] Using branch: master as version is unstable, use with caution
[INFO:swift] Loading the model using model_dir: /home/setup/.cache/modelscope/hub/models/qwen/Qwen2-VL-2B-Instruct
[INFO:swift] model_kwargs: {'device_map': 'auto'}
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.17it/s]
[INFO:swift] model.max_model_len: 32768
[INFO:swift] Global seed set to 42


In [2]:
#tuned 

import torch
from swift.llm import get_model_tokenizer, get_template, inference, ModelType,get_default_template_type
from swift.utils import seed_everything
from transformers import Qwen2VLForConditionalGeneration

#モデルのパス
local_model_path = "/data/2024/Chem_vision/250301ft_qwen/output/qwen2-vl-2b-instruct/v3-20250301-045328/checkpoint-183"
local_model_path="/data/2024/Chem_vision/250301ft_qwen/output/qwen2-vl-2b-instruct/v4-20250301-164042/checkpoint-183"
#モデル初期化
model_type = ModelType.qwen2_vl_2b_instruct
template_type = get_default_template_type(model_type)

#この方式でtokenizerのみ読み込み
_, tokenizer_ft = get_model_tokenizer(model_type, torch.bfloat16,
                                       load_model=False,
                                       model_kwargs={'device_map': 'auto',
                                                     })
# ローカルモデルの読み込み
model_ft = Qwen2VLForConditionalGeneration.from_pretrained(
    local_model_path, 
    torch_dtype=torch.bfloat16,
    device_map='auto',
)
model_ft.generation_config.max_new_tokens = 256
template = get_template(template_type, tokenizer_ft)
seed_everything(42)

#推論
#クエリを投げる



[INFO:swift] Downloading the model from ModelScope Hub, model_id: qwen/Qwen2-VL-2B-Instruct


[WARNING:modelscope] Using branch: master as version is unstable, use with caution
[INFO:swift] Loading the model using model_dir: /home/setup/.cache/modelscope/hub/models/qwen/Qwen2-VL-2B-Instruct
[INFO:swift] Global seed set to 42


42

In [3]:

path_list="""
/data/2024/Chem_vision/250301ft_qwen/dataset/catalogue/A/A_0097.png
"""
path_list = path_list.strip().split('\n')

for target_url in path_list:
    query = f"""<img>{target_url}</img>これはなんだい?"""
    response, history = inference(model, template, query)
    print(f'query: {query}')
    print(f'response: {response}')

print("ft")
for target_url in path_list:
    query = f"""<img>{target_url}</img>これはなんだい?"""
    response, history = inference(model_ft, template, query)
    print(f'query: {query}')
    print(f'response: {response}')

[INFO:swift] Setting image_factor: 28. You can adjust this hyperparameter through the environment variable: `IMAGE_FACTOR`.
[INFO:swift] Setting resized_height: None. You can adjust this hyperparameter through the environment variable: `RESIZED_HEIGHT`.
[INFO:swift] Setting resized_width: None. You can adjust this hyperparameter through the environment variable: `RESIZED_WIDTH`.
[INFO:swift] Setting min_pixels: 3136. You can adjust this hyperparameter through the environment variable: `MIN_PIXELS`.
[INFO:swift] Setting max_pixels: 12845056. You can adjust this hyperparameter through the environment variable: `MAX_PIXELS`.


query: <img>/data/2024/Chem_vision/250301ft_qwen/dataset/catalogue/A/A_0097.png</img>これはなんだい?
response: これは「アズワン」製のミルです。ミルは、液体を高速で回転させ、その中に含まれる微粒子を分散させる装置です。このミルは、化学実験や生物実験など、様々な用途に使用されます。
ft
query: <img>/data/2024/Chem_vision/250301ft_qwen/dataset/catalogue/A/A_0097.png</img>これはなんだい?
response: ミニマルクター(フロントタイプ) MCシリーズです｡
